### Importing Modules

In [1]:
#Import modules
import nsepy
from datetime import date
import numpy as np

### Reading Symbols from File

In [2]:
#Import pandas
import pandas as pd
#Import file and read
data=pd.read_csv("ind_nifty200list.csv")
#Let's look at data
data.head(5)

,Company Name,Industry,Symbol,Series,ISIN Code
0,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
1,AU Small Finance Bank Ltd.,FINANCIAL SERVICES,AUBANK,EQ,INE949L01017
2,Aarti Industries Ltd.,CHEMICALS,AARTIIND,EQ,INE769A01020
3,Abbott India Ltd.,PHARMA,ABBOTINDIA,EQ,INE358A01014
4,Adani Enterprises Ltd.,METALS,ADANIENT,EQ,INE423A01024


In [3]:
#symbols
symbols=list(data["Symbol"].values)

In [4]:
#Import dask
import dask

In [11]:
#Call api using list of symbols and start-end dates (Parallel Using Dask)
total_result=[]

for sym in symbols:
    sbin = dask.delayed(nsepy.get_history)(symbol=sym,start=date(2020,8,1),end=date(2021,6,25))
    total_result.append(sbin)
    
data_arr=dask.delayed()(total_result)

In [12]:
import time 
start=time.time()
total_history_dataset=data_arr.compute()
end=time.time()
print("Time for execution: {}".format(end-start))

Time for execution: 84.12635278701782


In [13]:
#Total symbols in the dataset generated
df=total_history_dataset
len(df)

200

###  PART 1   :    Calculate % change of stocks   (For last 10 days)

In [15]:
def change_per_symbol(idx):
    #Extracting close price  : inverting order
    cp=df[idx]["Close"].values[-20:][::-1]
    result=[]
    
    for i in range(10):
        close_today=cp[i]
        ten_days_data=cp[i:i+10]
        avg_moving_ten_days=np.mean(ten_days_data)
        
        #Calculate change
        change=(close_today-avg_moving_ten_days)/avg_moving_ten_days
        
        result.append(change)
    
    result=np.array(result)
    return np.mean(result)
    
    
total_result=[]
for i in range(len(df)):
    try:
        sym=df[i]["Symbol"].values[0]
        #Call function to calcuate change per symbol
        y=dask.delayed(change_per_symbol)(i)
        total_result.append((sym,y))
    except:
        pass

#Fetch result
z=dask.delayed()(total_result)

start=time.time()
#Compute dask function
result = z.compute()
end=time.time()

#Display
print("Time for execution: {} secs".format(end-start))

Time for execution: 0.04471230506896973 secs


In [16]:
#Result for change for first 20 symbols 
result[:20]

[('ACC', 0.0024391403579039498),
 ('AUBANK', 0.007770957422810344),
 ('AARTIIND', -0.17015102979910263),
 ('ABBOTINDIA', 0.01188482472835971),
 ('ADANIENT', -0.0301712165230835),
 ('ADANIGREEN', 0.001971553419495599),
 ('ADANIPORTS', -0.07568606619190674),
 ('ADANIGAS', 0.08198371774292615),
 ('ADANITRANS', 0.053734859503954346),
 ('ABCAPITAL', -0.01907286593419156),
 ('ABFRL', 0.011463882108126312),
 ('AJANTPHARM', 0.002741501292957587),
 ('APLLTD', 0.006128434534834514),
 ('ALKEM', -0.0029765935988459117),
 ('AMARAJABAT', -0.003929967935302688),
 ('AMBUJACEM', 0.01114696846383311),
 ('APOLLOHOSP', -0.006493712453195759),
 ('APOLLOTYRE', -0.0226564002816658),
 ('ASHOKLEY', -0.029315670728516752),
 ('ASIANPAINT', 0.013031894074491087)]

###  PART 2   :    Calculate % change of stocks   (For last 100 days)

In [17]:
def change_per_symbol(idx):
    #Extracting close price  : inverting order
    cp=df[idx]["Close"].values[-110:][::-1]
    result=[]
    
    for i in range(100):
        close_today=cp[i]
        ten_days_data=cp[i:i+10]
        avg_moving_ten_days=np.mean(ten_days_data)
        
        #Calculate change
        change=(close_today-avg_moving_ten_days)/avg_moving_ten_days
        
        result.append(change)
    
    result=np.array(result)
    return np.mean(result)
    
    
total_result=[]
for i in range(len(df)):
    try:
        sym=df[i]["Symbol"].values[0]
        #Call function to calcuate change per symbol
        y=dask.delayed(change_per_symbol)(i)
        total_result.append((sym,y))
    except:
        pass

#Fetch result
z=dask.delayed()(total_result)

start=time.time()
#Compute dask function
result = z.compute()
end=time.time()

#Display
print("Time for execution: {} secs".format(end-start))

Time for execution: 0.20891499519348145 secs


In [20]:
#Result for change for first 20 symbols 
result[:20]

[('ACC', 0.009037428952352189),
 ('AUBANK', 0.007287817701778332),
 ('AARTIIND', 0.0021424774579712437),
 ('ABBOTINDIA', 0.005746769032602292),
 ('ADANIENT', 0.049014570117527655),
 ('ADANIGREEN', 0.00845445467229479),
 ('ADANIPORTS', 0.014066147066034446),
 ('ADANIGAS', 0.06690344166541458),
 ('ADANITRANS', 0.058026283867146346),
 ('ABCAPITAL', 0.015968457876428463),
 ('ABFRL', 0.011443584988126039),
 ('AJANTPHARM', 0.0052233970155231655),
 ('APLLTD', 0.0009857267254703359),
 ('ALKEM', 0.0011052111988328894),
 ('AMARAJABAT', -0.011521466270630176),
 ('AMBUJACEM', 0.013799373308500777),
 ('APOLLOHOSP', 0.010118546063189958),
 ('APOLLOTYRE', 0.003970833313053356),
 ('ASHOKLEY', 0.001291925474113856),
 ('ASIANPAINT', 0.007865379623524363)]